In [1]:
import cv2
def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (cnts, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

In [2]:
test_image = cv2.imread("C:/Users/pytho/Desktop/maxresdefault.jpg")
segment(test_image)

NameError: name 'bg' is not defined

In [3]:
# Initialize background model
bg = None

# Capture video from camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur to reduce noise
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Set background image for first frame
    if bg is None:
        bg = gray.copy().astype("float")
        continue

    # Segment the hand region
    segmented = segment(gray)

    # If hand region is detected, draw contour on original frame
    if segmented is not None:
        (thresholded, segmented) = segmented
        x, y, w, h = cv2.boundingRect(segmented)
        right = x + w
        top = y + h
        cv2.drawContours(frame, [segmented], -1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow("Video Feed", frame)

    # Check for 'q' key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()